# Hospital Placement with K-Means

Notebook de soporte para analizar el dataset sintético de vecindarios,
explorar la demanda hospitalaria y entrenar un modelo K-Means que defina
posibles ubicaciones de hospitales.

## 1. Configuración inicial

Este notebook asume que el archivo `../data/neighborhoods_synthetic.csv` está
disponible (cuando se ejecuta desde el repositorio clonado). Si trabajas en
Colab y el archivo no existe localmente, se solicitará subirlo de forma manual.

In [ ]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

px.defaults.template = 'plotly_dark'
px.defaults.color_continuous_scale = 'Magma'
px.defaults.width = 900
px.defaults.height = 500

DATA_PATH = Path('../data/neighborhoods_synthetic.csv')

In [ ]:
def load_dataset(path: Path = DATA_PATH) -> pd.DataFrame:
    if path.exists():
        print(f'Leyendo dataset desde {path}')
        return pd.read_csv(path)
    print('Archivo no encontrado, intenta subirlo manualmente...')
    try:
        from google.colab import files  # type: ignore
        uploaded = files.upload()
        if not uploaded:
            raise FileNotFoundError('No se subió ningún archivo')
        name = next(iter(uploaded))
        print(f'Utilizando archivo subido: {name}')
        return pd.read_csv(name)
    except ModuleNotFoundError as exc:
        raise FileNotFoundError(
            'Ejecuta este notebook en Colab o coloca el CSV en data/'
        ) from exc

df = load_dataset()
df.head()

In [ ]:
df.describe(include='all').transpose()

## 2. Exploración visual (modo oscuro)

In [ ]:
fig = px.histogram(
    df,
    x='demand_index',
    nbins=30,
    title='Distribución de demanda hospitalaria',
    opacity=0.9,
    color_discrete_sequence=['#f72585'],
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x='population',
    y='demand_index',
    color='elderly_pct',
    title='Población vs índice de demanda',
    color_continuous_scale='Turbo',
    hover_data=['zone', 'chronic_disease_pct'],
    trendline='ols',
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()

In [ ]:
fig = px.scatter(
    df,
    x='x_km',
    y='y_km',
    color='zone',
    size='demand_index',
    symbol='zone',
    title='Mapa de vecindarios y su demanda',
    hover_data=['population', 'demand_index'],
)
fig.update_layout(
    margin=dict(l=40, r=20, t=60, b=40), xaxis_title='X (km)', yaxis_title='Y (km)'
)
fig.show()

## 3. Método del codo para elegir *k*

In [ ]:
X = df[['x_km', 'y_km']].values
ks = list(range(1, 11))
inertias = []
for k in ks:
    model = KMeans(n_clusters=k, n_init=20, random_state=2024)
    model.fit(X)
    inertias.append(model.inertia_)

fig = px.line(
    x=ks,
    y=inertias,
    markers=True,
    title='Curva del codo (inercia vs k)',
)
fig.update_layout(xaxis_title='Número de hospitales (k)', yaxis_title='Inercia')
fig.show()

## 4. Entrenamiento final de K-Means

In [ ]:
SELECTED_K = 4  # <-- Ajusta este valor tras analizar la curva del codo
print(f'Entrenando modelo final con k={SELECTED_K}')
final_model = KMeans(n_clusters=SELECTED_K, n_init=25, random_state=2024)
clusters = final_model.fit_predict(X)
df['cluster'] = clusters
centroids = final_model.cluster_centers_
centroids

In [ ]:
fig = px.scatter(
    df,
    x='x_km',
    y='y_km',
    color='cluster',
    title='Clusters resultantes y hospitales propuestos',
    color_continuous_scale='Plasma',
    hover_data=['zone', 'demand_index'],
)
fig.add_scatter(
    x=centroids[:, 0],
    y=centroids[:, 1],
    mode='markers',
    marker=dict(size=18, color='#f4d35e', symbol='x', line=dict(color='white', width=2)),
    name='Hospitales',
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()

In [ ]:
summary = (
    df.groupby('cluster')
    .agg({'id': 'count', 'demand_index': 'mean', 'population': 'mean'})
    .rename(
        columns={
            'id': 'neighborhoods',
            'demand_index': 'avg_demand',
            'population': 'avg_population',
        }
    )
)
summary

## 5. Exportar centroides como `pretrained_hospitals.json`

In [ ]:
output_path = Path('pretrained_hospitals.json')
payload = {
    'k': int(SELECTED_K),
    'hospitals': centroids.round(3).tolist(),
    'description': f'Modelo K-Means (k={SELECTED_K}) entrenado sobre {DATA_PATH.name}',
}
with output_path.open('w', encoding='utf-8') as fp:
    json.dump(payload, fp, ensure_ascii=False, indent=2)
print(f'Archivo guardado en {output_path.resolve()}')

In [ ]:
try:
    from google.colab import files  # type: ignore
    files.download(str(output_path))
except ModuleNotFoundError:
    print('Descarga manual disponible solo en Colab, copia el archivo desde esta carpeta.')